In [2]:
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import pprint

import datetime

In [3]:
client = MongoClient('localhost', 27017)

### Create Database / Use Existing Databsae

In [4]:
db = client['test']

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

### Create Collection / Use Existing Collection

In [6]:
collection = db['test']

In [7]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'test')

### Insert one

In [8]:
post = {"author": "Mike", 
        "text": "My first blog post!", 
        "tags": ["mongodb", "python", "pymongo"], 
        "date": datetime.datetime.utcnow()}

In [9]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
db.list_collection_names()

In [18]:
pprint.pprint(posts.find_one())

{'_id': ObjectId('5d6e2f29926f50844a87f180'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 9, 3, 9, 14, 30, 493000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [16]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('5d6e2f29926f50844a87f180'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 9, 3, 9, 14, 30, 493000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [19]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('5d6e2f29926f50844a87f180'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 9, 3, 9, 14, 30, 493000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


`NOTE`: An ObjectId is not the same as its string representation:

### Bulk Inserts

In [20]:
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # No result

In [23]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [24]:
result = posts.insert_many(new_posts)

In [25]:
result.inserted_ids

[ObjectId('5d6e315d926f50844a87f181'), ObjectId('5d6e315d926f50844a87f182')]

`NOTE`: MongoDB is schema-free.

### Querying for More Than One Document

In [49]:
for post in posts.find():
    pprint.pprint(post)

{'_id': ObjectId('5d6e2f29926f50844a87f180'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 9, 3, 9, 14, 30, 493000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5d6e30a42c27be51d573773c'), 'marks': 55.0, 'name': 'Ram'}
{'_id': ObjectId('5d6e315d926f50844a87f181'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5d6e315d926f50844a87f182'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5d6e415b926f50844a87f187'),
 'image': b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkS'
          b'Ew8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCABr'
          b'AGsBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgED'
          b'AwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2Jy'
          b'ggkKFhcY

### Counting

In [27]:
posts.count_documents({})

4

In [28]:
posts.count_documents({"author": "Mike"})

2

### Range Queries

In [30]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('5d6e315d926f50844a87f182'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5d6e315d926f50844a87f181'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


### Indexing

In [33]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

In [35]:
user_profiles = [{'user_id': 211, 'name': 'Luke'},{'user_id': 212, 'name': 'Ziltoid'}]
result = db.profiles.insert_many(user_profiles)

In [38]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile)  # This is fine.

In [39]:
result = db.profiles.insert_one(duplicate_profile) # DuplicateKeyError                         

DuplicateKeyError: E11000 duplicate key error collection: test.profiles index: user_id_1 dup key: { : 212 }

### Save Images in DB

In [53]:
import base64
from PIL import Image
from bson import Binary

In [54]:
with open("./binary_img.jpg", "rb") as imageFile:
    img_str = base64.b64encode(imageFile.read())
    print(img_str)

b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCABrAGsBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APc7PVtN1C4urey1C0uZ7R9lzHDMrtC2SMOAcqcqRg+h9KuUUUV4v+0jfW8fg3SbBpMXU2oedGm0/MiRuGOenBkT8/Y11nwYnmufhJoLzyySuElQM7FiFWZ1Uc9goAA7AAV3lFFYfiXxhoHg+3gn17Uo7NJ3KRAozs5AycKoJwOMnGBkeoq5fa5penaHJrd1fwR6YkQmN0H3IUONpUjO7ORjGc5AGc1j+CPHml+PdOub3Sre+hjt5fKcXcOzJwDwwJU9egORxkAEZ6ivkj4T6tceCfi1DYX1tiS4lbSLpBh2jdnAGCGxxIq5PPy7sZOK+t6KKK8n8WXX279oTwRpFzbwSWtraT3ke9MnzGSXk54+UwIRxkHnPTHqkEENrbxW9vFHDBEgSOONQqooGAABwABxipKKK8T/AGk9Nhl8K6NqjNJ59vem3RQRtKyIWYnjOcxLjnuevblPFHid9a/Zs0ARz7JLfUI9Nu44dyKRFHIUVs/e+UROeo3ehGB6H+z7ZLa/DITLLI5u72WZlaFkCEBUwpPDjCA7hxkleqmvVK+UPjJZ3H

In [55]:
post_data = {'image': img_str}
result = posts.insert_one(post_data)
print('One post: {0}'.format(result.inserted_id))

One post: 5d6e42cf926f50844a87f188


### Get back image from DB

In [56]:
img_str_ = posts.find_one(ObjectId('5d6e42cf926f50844a87f188'))

In [57]:



fh = open("test2.jpg", "wb")
fh.write(img_str_.decode('base64'))
fh.close()

AttributeError: 'dict' object has no attribute 'decode'